In [1]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
api_key = #"my_api_key"
baseurl = "http://api.weatherapi.com/v1"

In [3]:
def get_city_data(city, api_key):
    today = pd.Timestamp.now().strftime("%Y-%m-%d")

    # Current weather
    current_url = f"{baseurl}/current.json?key={api_key}&q={city}"
    current_response = requests.get(current_url)
    current = current_response.json()['current']

    # 3-day forecast
    forecast_url = f"{baseurl}/forecast.json?key={api_key}&q={city}&days=3"
    forecast_response = requests.get(forecast_url)
    forecast_data = forecast_response.json()['forecast']['forecastday']

    forecast_summary = []
    for day in forecast_data:
        date_obj = datetime.strptime(day['date'], '%Y-%m-%d')
        formatted_date = date_obj.strftime('%A, %B %#d')
        forecast_summary.append(
            f"{formatted_date}: High {day['day']['maxtemp_f']}F, Low {day['day']['mintemp_f']}F - {day['day']['condition']['text']}"
        )

    # Astronomy data
    astronomy_url = f"{baseurl}/astronomy.json?key={api_key}&q={city}&dt={today}"
    astronomy_response = requests.get(astronomy_url)
    astronomy = astronomy_response.json()['astronomy']['astro']

    # Summary dictionary
    summary = {
        'City': city,
        'Temperature (F)': current['temp_f'],
        'Condition': current['condition']['text'],
        'Humidity': current['humidity'],
        'Wind (mph)': current['wind_mph'],
        'Sunrise': astronomy['sunrise'],
        'Sunset': astronomy['sunset'],
        '3-Day Forecast': " | ".join(forecast_summary)
    }

    return summary

In [4]:
# Cities list
cities = ['Louisville', 'New York', 'Los Angeles', 'Chicago', 'Miami']

In [5]:
# Data for all cities
city_data_list = []
for city in cities:
    city_data = get_city_data(city, api_key)
    city_data_list.append(city_data)

In [6]:
#Dataframe
df = pd.DataFrame(city_data_list)

In [7]:
# Show the DataFrame
print(df)

          City  Temperature (F)      Condition  Humidity  Wind (mph)  \
0   Louisville             73.0          Clear        55         3.6   
1     New York             69.1  Partly cloudy        73         8.9   
2  Los Angeles             71.1  Partly cloudy        76         4.9   
3      Chicago             75.0          Clear        54         4.3   
4        Miami             81.0       Overcast        79        16.3   

    Sunrise    Sunset                                     3-Day Forecast  
0  07:27 AM  07:48 PM  Wednesday, September 17: High 93.2F, Low 67.1F...  
1  06:39 AM  07:01 PM  Wednesday, September 17: High 66.9F, Low 61.5F...  
2  06:38 AM  06:56 PM  Wednesday, September 17: High 85.3F, Low 73.0F...  
3  06:33 AM  06:56 PM  Wednesday, September 17: High 79.5F, Low 63.9F...  
4  07:08 AM  07:23 PM  Wednesday, September 17: High 86.5F, Low 76.6F...  


In [ ]:
# Save to CSV
df.to_csv("weather_summary.csv", index=False)